# AG2 Bug Reproduction: Empty Messages in Swarm

This notebook demonstrates a bug in AG2's swarm functionality where empty messages are passed to the initial agent when it's selected as the first speaker in a swarm conversation.

## Background

We've observed that when using `initiate_swarm_chat` from AG2, the initial agent is correctly selected as the next speaker but doesn't receive the initial message in its `generate_reply` method. This causes errors or unexpected behavior in the swarm conversation.

## Reproduction Steps

In [26]:
!pip install ag2  # Install AG2 framework

In [27]:
import logging
import sys
from typing import Dict, Any, List, Optional

# Setup logging to easily see the bug
logging.basicConfig(stream=sys.stdout, level=logging.INFO, 
                   format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("bug_repro")

# Import AG2 components
from autogen import (
    ConversableAgent,
    register_hand_off,
    AfterWork,
    AfterWorkOption,
    initiate_swarm_chat,
    OnCondition
)

## Create a Modified Agent Class

We'll create a derived agent class that logs when it receives empty messages in the generate_reply method.

In [28]:
class DebuggingAgent(ConversableAgent):
    """Agent class that logs details about the messages it receives."""
    
    def generate_reply(self, messages=None, sender=None):
        """Log information about the messages received and call parent method."""
        # Debug info
        logger.info(f"{self.name} received {len(messages) if messages else 0} messages")
        
        # Check for empty messages list - this indicates the bug
        if not messages or len(messages) == 0:
            logger.error(f"BUG DETECTED: Empty messages list passed to {self.name}.generate_reply")
            # Create a default message for Lead to avoid failure
            if self.name == "Lead":
                logger.warning(f"Creating default message for {self.name} to avoid empty messages error")
                messages = [{
                    "role": "user", 
                    "content": "Please provide initial guidance for this task."
                }]
        
        # Print the first message for debugging
        if messages and len(messages) > 0:
            logger.info(f"First message content: {messages[0].get('content', '')[0:50]}...")
        
        # Call parent implementation
        return super().generate_reply(messages, sender)

## Create Agent Team with Handoff Pattern

Now we'll create a team of agents with a simple handoff pattern to demonstrate the bug.

In [29]:
# Create team agents
lead = DebuggingAgent(
    name="Lead",
    system_message="You are a team leader that guides the overall project.",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo"}]}
)

researcher = DebuggingAgent(
    name="Researcher",
    system_message="You are a researcher who gathers information.",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo"}]}
)

developer = DebuggingAgent(
    name="Developer",
    system_message="You are a developer who implements solutions.",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo"}]}
)

# Configure handoffs between agents
register_hand_off(
    lead,
    [
        OnCondition(
            condition="Indicates a need for research or information gathering.",
            target=researcher
        ),
        OnCondition(
            condition="Indicates a need for implementation or coding.",
            target=developer
        ),
        AfterWork(agent=AfterWorkOption.TERMINATE)
    ]
)

register_hand_off(
    researcher,
    [AfterWork(agent=lead)]
)

register_hand_off(
    developer,
    [AfterWork(agent=lead)]
)

## Run the Swarm to Demonstrate the Bug

Now we'll run the swarm with the Lead agent as the initial agent to trigger the bug.

In [30]:
def run_swarm(query: str, initial_agent_name: str = "Lead"):
    """Run a swarm chat with the given query and initial agent.
    
    Args:
        query: The query to start the conversation with
        initial_agent_name: Name of the agent to start as speaker
    """
    logger.info(f"Starting swarm with initial agent {initial_agent_name}")
    
    # Select the initial agent
    if initial_agent_name == "Lead":
        initial_agent = lead
    elif initial_agent_name == "Researcher":
        initial_agent = researcher
    elif initial_agent_name == "Developer":
        initial_agent = developer
    else:
        raise ValueError(f"Unknown agent name: {initial_agent_name}")
    
    # List of all agents
    all_agents = [lead, researcher, developer]
    
    # Prepare initial message
    # initial_message = {
    #     "role": "user", 
    #     "content": f"Task: {query}\n\nPlease analyze this task and coordinate with the team to complete it."
    # }
    initial_message = f"Task: {query}\n\nPlease analyze this task and coordinate with the team to complete it."
    
    # logger.info(f"Initial message prepared: {initial_message['content'][0:50]}...")
    
    try:
        # Run the swarm
        chat_result = initiate_swarm_chat(
            initial_agent=initial_agent,
            agents=all_agents,
            messages=initial_message,  # We provide a valid initial message
            max_rounds=5,
            after_work=AfterWorkOption.TERMINATE
        )
        
        # Return summary of results
        if isinstance(chat_result, tuple) and len(chat_result) > 0:
            return {
                "success": True,
                "chat_history": chat_result[0].chat_history if hasattr(chat_result[0], 'chat_history') else []
            }
        else:
            return {"success": True, "chat_result": chat_result}
        
    except Exception as e:
        import traceback
        logger.error(f"Error in swarm execution: {str(e)}")
        logger.error(traceback.format_exc())
        return {"success": False, "error": str(e)}

In [31]:
# Run the swarm with Lead as initial agent to demonstrate the bug
result = run_swarm("Create a simple web application using Flask")

# Print summary
print("\n\nSWARM EXECUTION COMPLETED")
print(f"Success: {result['success']}")

if 'error' in result:
    print(f"Error: {result['error']}")

2025-04-11 08:25:21,507 - bug_repro - INFO - Starting swarm with initial agent Lead
_User (to chat_manager):

Task: Create a simple web application using Flask

Please analyze this task and coordinate with the team to complete it.

--------------------------------------------------------------------------------

Next speaker: Lead

2025-04-11 08:25:21,512 - bug_repro - INFO - Lead received 0 messages
2025-04-11 08:25:21,513 - bug_repro - ERROR - BUG DETECTED: Empty messages list passed to Lead.generate_reply
2025-04-11 08:25:21,514 - bug_repro - WARNING - Creating default message for Lead to avoid empty messages error
2025-04-11 08:25:21,514 - bug_repro - INFO - First message content: Please provide initial guidance for this task....

>>>>>>>> USING AUTO REPLY...
Lead (to chat_manager):

Sure, let's break down the task into smaller steps:

1. Research existing API options for weather information.
2. Evaluate different APIs based on their features, pricing, and ease of use.
3. Select th

## Workaround Demonstration

To contrast, let's try running with a different initial agent to see if the bug still occurs.

In [32]:
# Run the swarm with Researcher as initial agent for comparison
result2 = run_swarm("Create a simple web application using Flask", initial_agent_name="Researcher")

# Print summary
print("\n\nSWARM EXECUTION COMPLETED (ALTERNATIVE APPROACH)")
print(f"Success: {result2['success']}")

if 'error' in result2:
    print(f"Error: {result2['error']}")

2025-04-11 08:25:21,614 - bug_repro - INFO - Starting swarm with initial agent Researcher
_User (to chat_manager):

Task: Create a simple web application using Flask

Please analyze this task and coordinate with the team to complete it.

--------------------------------------------------------------------------------

Next speaker: Researcher

2025-04-11 08:25:21,617 - bug_repro - INFO - Researcher received 0 messages
2025-04-11 08:25:21,617 - bug_repro - ERROR - BUG DETECTED: Empty messages list passed to Researcher.generate_reply

>>>>>>>> USING AUTO REPLY...
Researcher (to chat_manager):

Sure, I'll need more information to assist you better. Can you provide details about the requirements for the simple web application that you want to create using Flask? For example, what specific features should the web application have, who is the target audience, and what is the expected timeline for completion?

--------------------------------------------------------------------------------

N

## Bug Summary

The bug observed is:

1. When using `initiate_swarm_chat` with `initial_agent` set to a specific agent, that agent is correctly selected as the first speaker
2. However, when the initial agent's `generate_reply` method is called, it receives an empty messages list (or messages=None)
3. This happens despite properly providing an initial message in the `messages` parameter to `initiate_swarm_chat`
4. This bug only occurs for the first agent in the conversation

## Workaround

Our current workaround is to special-case the initial agent (in our case, Lead) to handle empty messages lists by creating a default message. However, this is not ideal as it masks the underlying bug in the AG2 framework.